In [1]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import metrics
import myfm
from myfm import RelationBlock
import pandas as pd
from scipy import sparse as sps
from mapper import DefaultMapper

# read movielens 100k data.
from movielens1M_data import MovieLens1MDataManager

In [2]:
data_manager = MovieLens1MDataManager()
df_train, df_test = data_manager.load_rating()

user_to_internal = DefaultMapper(df_train.user_id.values)
movie_to_internal = DefaultMapper(df_train.movie_id)

## Implement flavor of SVD++

We add "all users who have evaluated a movie in the train set" as the movie's feature.

In [3]:
movie_vs_watched = dict()
user_vs_watched = dict()
for row in df_train.itertuples():
    user_id = row.user_id
    movie_id = row.movie_id
    movie_vs_watched.setdefault(movie_id, list()).append(user_id)
    user_vs_watched.setdefault(user_id, list()).append(movie_id)

In [18]:
use_ii = True
use_iu = True

In [19]:
feature_group_sizes = [
    len(user_to_internal) # user ids
] + ([len(movie_to_internal)] if use_iu else [] ) +[
    len(movie_to_internal), # movie ids
] + ([len(user_to_internal)] if use_ii else [])# all users who watched the movies] e

grouping = [ i for i, size in enumerate(feature_group_sizes) for _ in range(size)]

In [20]:
# given user/movie ids, add additional infos and return it as sparse
def augment_user_id(user_ids):
    X = sps.lil_matrix((len(user_ids), len(user_to_internal) + (len(movie_to_internal) if use_iu else 0) ))
    for index, user_id in enumerate(user_ids):
        X[index, user_to_internal[user_id]] = 1
        if not use_iu:
            continue
        watched_movies = user_vs_watched.get(user_id, [])
        normalizer = 1 / max(len(watched_movies), 1) ** 0.5
        for mid in watched_movies:
            X[index, movie_to_internal[mid] + len(user_to_internal)] = normalizer
    return X.tocsr()

def augment_movie_id(movie_ids):
    X = sps.lil_matrix((len(movie_ids), len(movie_to_internal)+ (len(user_to_internal) if use_ii else 0 )))
    for index, movie_id in enumerate(movie_ids):
        X[index, movie_to_internal[movie_id]] = 1
        if not use_ii:
            continue
        watched_users = movie_vs_watched.get(movie_id, [])
        normalizer = 1 / max(len(watched_users), 1) ** 0.5
        for uid in watched_users:
            X[index, user_to_internal[uid] + len(movie_to_internal)] = normalizer
    return X.tocsr()

## User Relation Block to express data
See [\[Rendle 2013\]](http://www.vldb.org/pvldb/vol6/p337-rendle.pdf) how comlexity dcrease drastically in this case (and most cases with bipartite graph structure).

In [21]:
# Create RelationBlock.
train_blocks = []
test_blocks = []
for source, target in [(df_train, train_blocks), (df_test, test_blocks)]:
    unique_users, user_map = np.unique(source.user_id, return_inverse=True)
    target.append(
        RelationBlock(user_map, augment_user_id(unique_users))
    )
    unique_movies, movie_map = np.unique(source.movie_id, return_inverse=True)
    target.append(
        RelationBlock(movie_map, augment_movie_id(unique_movies))
    )

In [15]:
fm = myfm.MyFMRegressor(rank=10)
fm.fit(None, df_train.rating.values, X_rel=train_blocks,
        grouping=grouping,
        n_kept_samples=95, n_iter=100);

alpha = 1.58 w0 = 3.46 : 100%|██████████| 100/100 [00:35<00:00,  2.80it/s]


In [16]:
test_predictions = fm.predict(None, test_blocks, n_workers=4)

In [17]:
rmse = (
    (test_predictions - df_test.rating.values)**2
).mean() ** 0.5
mae = np.abs(test_predictions - df_test.rating).mean()

# Note the improvement from "id_only" case.
# Compare this with methods like ones in https://paperswithcode.com/sota/collaborative-filtering-on-movielens-100k
print('rmse={}, mae={}'.format(rmse, mae))

rmse=0.8449757354354077, mae=0.6637632643658582
